In [1]:
data_dir = '/home/liujing/ForTitanServer/KaggleDSB_raw/preprocess/755_clean.npy'
bboxes = '/home/liujing/ForTitanServer/KaggleDSB_raw/preprocess/755_label.npy'

In [2]:
import numpy as np
imgs = np.load(data_dir)
stride = 4
pad_value = 170

In [3]:
nz, nh, nw = imgs.shape[1:]

In [4]:
imgs.shape[1:]

(309, 191, 248)

In [5]:
pz = int(np.ceil(float(nz) / stride)) * stride
ph = int(np.ceil(float(nh) / stride)) * stride
pw = int(np.ceil(float(nw) / stride)) * stride

In [6]:
pz

312

In [7]:
imgs = np.pad(imgs, [[0,0],[0, pz - nz], [0, ph - nh], [0, pw - nw]], 'constant',constant_values = pad_value)

In [8]:
imgs.shape

(1, 312, 192, 248)

In [9]:
xx,yy,zz = np.meshgrid(np.linspace(-0.5,0.5,imgs.shape[1]/4),
                                   np.linspace(-0.5,0.5,imgs.shape[2]/4),
                                   np.linspace(-0.5,0.5,imgs.shape[3]/4),indexing ='ij')

In [10]:
xx.shape

(78, 48, 62)

In [11]:
yy.shape

(78, 48, 62)

In [12]:
zz.shape

(78, 48, 62)

In [13]:
coord = np.concatenate([xx[np.newaxis,...], yy[np.newaxis,...],zz[np.newaxis,:]],0).astype('float32')

In [14]:
coord.shape

(3, 78, 48, 62)

In [15]:
data = imgs
side_len = 144
max_stride = 16
stride = 4
margin = 32
      
assert(side_len > margin)
assert(side_len % max_stride == 0)
assert(margin % max_stride == 0)

splits = []
_, z, h, w = data.shape

nz = int(np.ceil(float(z) / side_len))
nh = int(np.ceil(float(h) / side_len))
nw = int(np.ceil(float(w) / side_len))
        
nzhw = [nz,nh,nw]
        
pad = [ [0, 0],
        [margin, nz * side_len - z + margin],
        [margin, nh * side_len - h + margin],
        [margin, nw * side_len - w + margin]]
data = np.pad(data, pad, 'edge')

for iz in range(nz):
    for ih in range(nh):
        for iw in range(nw):
            sz = iz * side_len
            ez = (iz + 1) * side_len + 2 * margin
            sh = ih * side_len
            eh = (ih + 1) * side_len + 2 * margin
            sw = iw * side_len
            ew = (iw + 1) * side_len + 2 * margin

            split = data[np.newaxis, :, sz:ez, sh:eh, sw:ew]
            splits.append(split)

splits = np.concatenate(splits, 0)
#return splits,nzhw

In [16]:
splits.shape

(12, 1, 208, 208, 208)

In [17]:
nzhw

[3, 2, 2]

In [18]:
data.shape

(1, 496, 352, 352)

In [19]:
range(nz)

[0, 1, 2]

In [20]:
def combine(self, output, nzhw = None, side_len=None, stride=None, margin=None):
        
        if side_len==None:
            side_len = self.side_len
        if stride == None:
            stride = self.stride
        if margin == None:
            margin = self.margin
        if nzhw is None:
            nz = self.nz
            nh = self.nh
            nw = self.nw
        else:
            nz,nh,nw = nzhw
        assert(side_len % stride == 0)
        assert(margin % stride == 0)
        side_len /= stride
        margin /= stride

        splits = []
        for i in range(len(output)):
            splits.append(output[i])

        output = -1000000 * np.ones((
            nz * side_len,
            nh * side_len,
            nw * side_len,
            splits[0].shape[3],
            splits[0].shape[4]), np.float32)

        idx = 0
        for iz in range(nz):
            for ih in range(nh):
                for iw in range(nw):
                    sz = iz * side_len
                    ez = (iz + 1) * side_len
                    sh = ih * side_len
                    eh = (ih + 1) * side_len
                    sw = iw * side_len
                    ew = (iw + 1) * side_len

                    split = splits[idx][margin:margin + side_len, margin:margin + side_len, margin:margin + side_len]
                    output[sz:ez, sh:eh, sw:ew] = split
                    idx += 1

        return output 

In [21]:
def split(data, side_len = None, max_stride = None, margin = None):
        
        
        assert(side_len > margin)
        assert(side_len % max_stride == 0)
        assert(margin % max_stride == 0)

        splits = []
        _, z, h, w = data.shape

        nz = int(np.ceil(float(z) / side_len))
        nh = int(np.ceil(float(h) / side_len))
        nw = int(np.ceil(float(w) / side_len))
        
        nzhw = [nz,nh,nw]
        
        pad = [ [0, 0],
                [margin, nz * side_len - z + margin],
                [margin, nh * side_len - h + margin],
                [margin, nw * side_len - w + margin]]
        data = np.pad(data, pad, 'edge')

        for iz in range(nz):
            for ih in range(nh):
                for iw in range(nw):
                    sz = iz * side_len
                    ez = (iz + 1) * side_len + 2 * margin
                    sh = ih * side_len
                    eh = (ih + 1) * side_len + 2 * margin
                    sw = iw * side_len
                    ew = (iw + 1) * side_len + 2 * margin

                    split = data[np.newaxis, :, sz:ez, sh:eh, sw:ew]
                    splits.append(split)

        splits = np.concatenate(splits, 0)
        return splits,nzhw

In [22]:
coord2, nzhw2 = split(coord,
                      side_len = side_len/stride,
                      max_stride = max_stride/stride,
                      margin = 8)

In [23]:
coord2.shape

(12, 3, 52, 52, 52)

In [24]:
nzhw2

[3, 2, 2]

In [25]:
assert np.all(nzhw==nzhw2)

In [26]:
splits = (splits.astype(np.float32)-128)/128

In [27]:
import torch
data = torch.from_numpy(splits.astype(np.float32))
target = bboxes
coord = torch.from_numpy(coord2.astype(np.float32))
nzhw = np.array(nzhw)

In [28]:
#data, target, coord, nzhw
nzhw[0]

3

In [29]:
data


( 0 , 0 , 0 ,.,.) = 
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
           ...             ⋱             ...          
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281

( 0 , 0 , 1 ,.,.) = 
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
           ...             ⋱             ...          
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281

( 0 , 0 , 2 ,.,.) = 
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281
  0.3281  0.3281  0.3281  ...   0.3281  0.3281  0.3281